# Import Library

- `selenium` for Browser Automation
- `bs4` for parse the HTML page
- `dotenv` to read env variable

In [112]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import time
from dotenv import load_dotenv
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [113]:
def load_env():
    load_dotenv()
    account_username = os.getenv('LEETCODE_USERNAME')
    account_password = os.getenv('LEETCODE_PASSWORD')

    return {
        'username': account_username,
        'password': account_password,
    }
    
env_var = load_env()

# Go to Login Page

In [114]:
driver = webdriver.Firefox(executable_path="./geckodriver.exe")

driver.get('https://leetcode.com/accounts/login/')
try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#id_login"))
    )
except Exception as err:
    print("Error: ", err)

# Fill Username and Password

In [115]:
username = driver.find_element_by_css_selector('#id_login')
username.send_keys(env_var['username'])
password = driver.find_element_by_css_selector('#id_password')
password.send_keys(env_var['password'])

# Click Login Button

In [116]:
time.sleep(10)
login_button = driver.find_element_by_css_selector('.btn-content-container__2HVS')
login_button.click()
time.sleep(10)

# Find All Accepted Submissions

In [ ]:
def find_accepted_submissions_link(table):
    rows = table.find_all('tr')
    submissions_link = []

    for row in rows:
        tds = row.find_all('td')
        if len(tds) == 0:
            continue
        
        submission_status = tds[2]
        is_accepted = submission_status.get_text() == "Accepted"
        submission_link = submission_status.find('a', href=True)

        if is_accepted:
            submissions_link.append("https://leetcode.com" + submission_link['href'])
    
    return submissions_link

accepted_submissions_link = []
page_num = 1   
while True:
    driver.get(f"https://leetcode.com/submissions/#/{page_num}")
    time.sleep(10)

    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    if soup.find_all(text="No more submissions."):
        break
    
    table = soup.find(id="submission-list-app").find_next("table")
    accepted_submissions_link += find_accepted_submissions_link(table)

    page_num += 1

# Save Accepted Submissions Link

In [122]:
with open("ac_submission_link.txt", "w") as f:
    for ac in accepted_submissions_link:
        f.write(f"{ac}\n")

# Parse Submission Data

In [ ]:
def parse_submission_data(page):
    soup = BeautifulSoup(page, 'html.parser')
    codes = soup.find("div", {"class": "ace_content"})

    first = True
    text_codes = []
    for child in codes.find_all("div"):
        if first and child.get_text()!= "":
            first = False
            continue
        if len(text_codes) > 0 and child.get_text() == text_codes[len(text_codes)-1]:
            continue
        text_codes.append(child.get_text())

    plain_text_code = '\n'.join(text_codes)
    language = soup.find(id="result_language").text
    return {
        'language': language,
        'plain_text_code': plain_text_code
    }

submissions_data = {}
for link in accepted_submissions_link:
    driver.get(link)
    time.sleep(10)

    problem_name = driver.title
    page = driver.page_source
    submission_data = parse_submission_data(page)
    if problem_name not in submissions_data:
        submissions_data[problem_name] = [submission_data]
    else:
        submissions_data[problem_name].append(submission_data)

print(submissions_data)

# Save Submissions Data

In [152]:
import json

with open("submissions_data.json", "w") as f:
    json.dump(submissions_data, f)